In [1]:
import os

In [4]:
%pwd

'c:\\Users\\08you\\Desktop\\My own 3 hour project\\Securing-Medical-Data-Transfer-Using-Deep-Steganography-and-Cryptography-MLflow-DVC'

In [3]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    Cover_train: Path
    Med_train: Path
    Cover_val:Path
    Med_val:Path
    trained_model_path: Path
    base_model_path: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import numpy as np
import tensorflow as tf

In [23]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        Med_train = training.Med_train
        Cover_train= training.Cover_train
        Med_val=training.Med_val
        Cover_val=training.Cover_val
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            base_model_path=Path(prepare_base_model.base_model_path),
            Med_train=Path(Med_train),
            Cover_train= Path(Cover_train),
            Med_val= Path(Med_val),
            Cover_val= Path(Cover_val),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [24]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from src.cnnClassifier.utils.common import custom_loss_1,custom_loss_2
import cv2
import time
import glob

In [27]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.TRAIN_NUM=len(glob.glob(str(config.Med_train)+"/*/*"))
        self.VAL_NUM=len(glob.glob(str(config.Med_val)+"/*/*"))
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.base_model_path,custom_objects={'custom_loss_2': custom_loss_2,'custom_loss_1': custom_loss_1}
        )
    
    # Function to convert grayscale image to RGB
    @staticmethod
    def gray_to_rgb(image):
        return cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    # Preprocessing functions
    @staticmethod
    def normalize_batch(imgs):
        '''Performs channel-wise z-score normalization'''

        return (imgs -  np.array([0.485, 0.456, 0.406])) /np.array([0.229, 0.224, 0.225])
    @staticmethod
    def denormalize_batch(imgs,should_clip=True):
        '''Denormalize the images for prediction'''

        imgs= (imgs * np.array([0.229, 0.224, 0.225])) + np.array([0.485, 0.456, 0.406])

        if should_clip:
            imgs= np.clip(imgs,0,1)
        return imgs

    # Custom generator for loading training images from directory
    def generate_generator_multiple(self,generator, med_path, cover_path):
        genX1 = generator.flow_from_directory(med_path, target_size=(224, 224), batch_size=self.config.params_batch_size, shuffle=True, class_mode=None)
        genX2 = generator.flow_from_directory(cover_path, target_size=(224, 224), batch_size=self.config.params_batch_size, shuffle=True, class_mode=None)

        while True:
            X1i = self.normalize_batch(genX1.next())
            X2i = self.normalize_batch(genX2.next())

            # Check if the images are grayscale, and convert to RGB if necessary
            if X1i.shape[-1] != 3:
                X1i = self.gray_to_rgb(X1i)
            if X2i.shape[-1] != 3:
                X2i = self.gray_to_rgb(X2i)

            yield ({'secret': X1i, 'cover': X2i}, {'hide_conv_f': X2i, 'revl_conv_f': X1i})

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.generate_generator_multiple(generator=valid_datagenerator, med_path=self.config.Med_val,cover_path=self.config.Cover_val)
        #needs to be two just like stego requirement
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = self.generate_generator_multiple(generator=train_datagenerator, med_path=self.config.Med_train,cover_path=self.config.Cover_train)
        self.validation_generator = self.generate_generator_multiple(generator=valid_datagenerator, med_path=self.config.Med_val,cover_path=self.config.Cover_val)
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    def train(self):
        self.steps_per_epoch = self.TRAIN_NUM// self.config.params_batch_size
        self.validation_steps = self.VAL_NUM // self.config.params_batch_size

        self.model.fit(
            self.train_generator,
            validation_data=self.validation_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
    

In [28]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-02-11 14:53:11,705: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-11 14:53:11,709: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-11 14:53:11,711: INFO: common: created directory at: artifacts]
[2024-02-11 14:53:11,713: INFO: common: created directory at: artifacts\data_ingestion\Data-CS-DVC\training_data]
Found 100 images belonging to 1 classes.
Found 100 images belonging to 1 classes.
6/6 [==============================] - ETA: 0s - loss: 2.4359 - hide_conv_f_loss: 1.6092 - revl_conv_f_loss: 1.1023  Found 50 images belonging to 1 classes.
Found 50 images belonging to 1 classes.
6/6 [==============================] - 825s 137s/step - loss: 2.4359 - hide_conv_f_loss: 1.6092 - revl_conv_f_loss: 1.1023 - val_loss: 2.3664 - val_hide_conv_f_loss: 1.5116 - val_revl_conv_f_loss: 1.1397
